In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import SparkSession
 
import pandas as pd
from pandas import Series
import numpy as np
import requests
import csv
import matplotlib.pyplot as plt
import warnings
from matplotlib import dates
import matplotlib as mlp
%matplotlib inline
 
from pyspark.sql.types import StructType,StructField, StringType
from pyspark import SparkContext
from pyspark.sql import SQLContext 
sql = SQLContext(sc)

In [0]:
from pyspark.sql import DataFrameStatFunctions
from pyspark.sql.functions import *
from pyspark import SparkContext
dbutils.library.installPyPI("missingpy", version="0.2.0")

Out[5]: True

In [0]:
%sql
select * from stocks_data limit 2

ticker,Sector,longBusinessSummary,fullTimeEmployees,Industry,shortName,environmentScore,governanceScore,socialScore,regularMarketOpen,twoHundredDayAverage10,regularMarketDayHigh,regularMarketDayLow,regularMarketPreviousClose,regularMarketVolume,Currency,previousClose,forwardPE17,averageVolume,averageDailyVolume10Day,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiftyDayAverage,twoHundredDayAverage23,payoutRatio24,earningsQuarterlyGrowth25,fiveYearAvgDividendYield,dividendRate27,marketCap,beta,pegRatio,Volume,dividendYield,enterpriseToRevenue,enterpriseToEbitda,forwardEps,sharesOutstanding,bookValue,heldPercentInsiders,floatShares,earningsQuarterlyGrowth40,shortPercentOfFloat,sharesShortPriorMonth,profitMargins,trailingAnnualDividendYield,trailingAnnualDividendRate,payoutRatio46,dividendRate47,trailingPE,forwardPE49,trailingEps,shortRatio,Goldman Sachs,JP Morgan,Morgan Stanley,Barclays,UBS,Bank of America,Credit Suisse
A,Healthcare,"Agilent Technologies, Inc. provides application focused solutions to the life sciences, diagnostics, and applied chemical markets worldwide. It operates in three segments: Life Sciences and Applied Markets, Diagnostics and Genomics, and Agilent CrossLab. The Life Sciences and Applied Markets segment offers liquid and gas chromatography systems and components; liquid and gas chromatography mass spectrometry systems; inductively coupled plasma mass spectrometry instruments; atomic absorption instruments; microwave plasma-atomic emission spectrometry instruments; inductively coupled plasma optical emission spectrometry instruments; raman spectroscopy; cell analysis plate based assays; flow cytometer; real-time cell analyzer; cell imaging systems; microplate readers; laboratory software and information management and analytics; laboratory automation and robotic systems; dissolution testing; vacuum pumps; and measurement technologies. The Diagnostics and Genomics segment provides reagents, instruments, software, and consumables; arrays for DNA mutation detection, genotyping, gene copy number determination, identification of gene rearrangements, DNA methylation profiling, and gene expression profiling, as well as sequencing target enrichment, genetic data management, and interpretation support software; and equipment focused on production of synthesized oligonucleotides for use as active pharmaceutical ingredients. The Agilent CrossLab segment offers GC and LC columns, sample preparation products, custom chemistries, and laboratory instrument supplies; and startup, operational, training, compliance support, and software as a service, as well as asset management and consultation services. The company markets its products through direct sales, distributors, resellers, manufacturer's representatives, and electronic commerce. It has collaboration agreement with SGS AXYS. The company was founded in 1999 and is headquartered in Santa Clara, California.",16300,Diagnostics & Research,"Agilent Technologies, Inc.",0.87,6.45,9.4,108.64,96.17864,112.3,108.315,108.68,2700586,USD,108.68,30.133152000000003,1260625,1769771,117.5,61.13,106.65143,96.17864,0.3171,0.042,0.88,0.78,3.4188496896E10,1.032694,3.58,2700586,0.006999999999999999,6.766,26.772,3.68,3.08310016E8,16.142,0.00381,307218219,0.042,0.010199999,2620892,0.13229999,0.006477733,0.7040000000000001,0.3171,0.78,50.063206,30.133152000000003,2.215,3.08,main,main,main,main,main,down,init
AAL,Industrials,"American Airlines Group Inc., through its subsidiaries, operates as a network air carrier. It provides scheduled air transportation services for passengers and cargo. As of December 31, 2019, the company operated a mainline fleet of 942 aircraft. It serves 365 destinations in approximately 61 countries, principally from its hubs and gateways in Charlotte, Chicago, Dallas/Fort Worth, London Heathrow, Los Angeles, Miami, New York, Philadelphia, Phoenix, and Washington, D.C. American Airlines Group Inc. has strategic partnership with JetBlue Airways Corp. The company was formerly kno

In [0]:
display(sqlContext.sql("select sector, count(industry) as number_companies from stocks_data group by sector order by number_companies desc"))

sector,number_companies
Financial Services,63
Industrials,61
Technology,60
Consumer Cyclical,60
Healthcare,57
Consumer Defensive,33
Real Estate,28
Utilities,27
Energy,22
Basic Materials,18


In [0]:
%sql
SELECT count(ticker) as tick,
CASE
    WHEN pegRatio > 1  THEN 'Overvalued'
    WHEN pegRatio = 1  THEN 'Reasonable'
    else 'Undervalued'
END AS stockvalue
FROM stocks_data
group by stockvalue;

tick,stockvalue
325,Overvalued
3,Reasonable
118,Undervalued


In [0]:
%sql
SELECT sector,avg(pegRatio) as Peg_Ratio,
CASE
    WHEN avg(pegRatio) > 1  THEN 'Overvalued'
    WHEN avg(pegRatio) < 1 THEN 'Undervalued'
END AS avg_pegRatio
FROM stocks_data
group by sector;

sector,Peg_Ratio,avg_pegRatio
Consumer Defensive,3.3278787878787877,Overvalued
Financial Services,1.804761904761904,Overvalued
Communication Services,0.33705882352941197,Undervalued
Energy,23.020000000000003,Overvalued
Healthcare,5.4359649122807,Overvalued
Real Estate,-0.8992857142857132,Undervalued
Utilities,2.587037037037037,Overvalued
Industrials,-0.6177049180327869,Undervalued
Technology,2.545833333333333,Overvalued
Consumer Cyclical,3.9595000000000007,Overvalued


In [0]:
display(sqlContext.sql("select avg(marketCap) as avg_marketCap,sector from stocks_data group by sector order by avg_marketCap desc "))

avg_marketCap,sector
2.1022948187482352E11,Communication Services
1.1818736976213333E11,Technology
7.494630282472728E10,Consumer Defensive
6.918175954526315E10,Healthcare
5.993492977173333E10,Consumer Cyclical
5.6989842114031746E10,Financial Services
3.839874383842623E10,Industrials
3.215405632474074E10,Utilities
2.9968119639272728E10,Energy
2.8185825792E10,Basic Materials


In [0]:
display(sqlContext.sql("select avg(profitMargins) as profit_margin,sector from stocks_data group by sector order by profit_margin desc"))

profit_margin,sector
0.20436071646428572,Real Estate
0.19368714288888889,Financial Services
0.18432116551666666,Technology
0.15037736707017546,Healthcare
0.13366296425925922,Utilities
0.09138393969696969,Consumer Defensive
0.0893981973770492,Industrials
0.08460176352941176,Communication Services
0.06327388833333335,Basic Materials
-0.009618833716666664,Consumer Cyclical


In [0]:
display(sqlContext.sql("select avg(marketCap) as avg,sector from stocks_data group by sector order by avg desc "))

avg,sector
2.1022948187482352E11,Communication Services
1.1818736976213333E11,Technology
7.494630282472728E10,Consumer Defensive
6.918175954526315E10,Healthcare
5.993492977173333E10,Consumer Cyclical
5.6989842114031746E10,Financial Services
3.839874383842623E10,Industrials
3.215405632474074E10,Utilities
2.9968119639272728E10,Energy
2.8185825792E10,Basic Materials


In [0]:
display(sqlContext.sql("select sector,count(ticker) as t from stocks_data group by sector order by t desc"))

sector,t
Financial Services,63
Industrials,61
Technology,60
Consumer Cyclical,60
Healthcare,57
Consumer Defensive,33
Real Estate,28
Utilities,27
Energy,22
Basic Materials,18


In [0]:
display(sqlContext.sql("select avg(regularMarketVolume) as reg,sector4 from stocks_f_data group by sector4 order by reg desc"))

reg,sector4
1.3966515E7,0
8417429.181818182,Energy
8163936.352941177,Communication Services
5286440.683333334,Consumer Cyclical
4907443.766666667,Technology
4313445.147540984,Industrials
4312522.65625,Financial Services
3876798.882352941,Consumer Defensive
3711792.814814815,Utilities
3632423.5263157897,Healthcare
